In [704]:
import pickle
import requests
from bs4 import BeautifulSoup
import time
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
import time
import os
import pandas as pd
import numpy as np
import statsmodels.api as sm
import statsmodels.formula.api as smf
import patsy
import diagnostic_plots
import unicodedata

import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.linear_model import RidgeCV
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import PolynomialFeatures
from sklearn.cross_validation import KFold
%matplotlib inline

with open('first_group_top40.pkl', 'rb') as picklefile:
    df = pickle.load(picklefile)
    
with open('tigers_pads_mets_marlins_wsox_top40.pkl', 'rb') as picklefile:
    tigers_pads_mets_marlins_wsox_df = pickle.load(picklefile)
       
with open('twins_red_rags_top40.pkl', 'rb') as picklefile:
    twins_red_rags_df = pickle.load(picklefile)

with open('big_group_top40.pkl', 'rb') as picklefile:
    big_group_df = pickle.load(picklefile)


In [705]:
df = pd.concat([df, tigers_pads_mets_marlins_wsox_df])

In [706]:
df = pd.concat([df, twins_red_rags_df])

In [707]:
df = pd.concat([df, big_group_df])

In [708]:
float_cols = ['BA', 'OBP', 'SLG', 'OPS', 'dWAR']
integer_cols = ['Year', 'Age', 'Games_Played', 'Plate_Appearances', 'At_Bats',
                'Runs', 'Hits', '2B', '3B', 'HR', 'RBI', 'SB', 'BB', 'SO', 'TB', 'HBP', 'SF']
for i in integer_cols:
    df[i] = df[i].astype(int)

df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 5111 entries, 0 to 2270
Data columns (total 33 columns):
Player_Name          5111 non-null object
Year                 5111 non-null int64
Age                  5111 non-null int64
Team                 5111 non-null object
League               5111 non-null object
Games_Played         5111 non-null int64
Plate_Appearances    5111 non-null int64
At_Bats              5111 non-null int64
Runs                 5111 non-null int64
Hits                 5111 non-null int64
2B                   5111 non-null int64
3B                   5111 non-null int64
HR                   5111 non-null int64
RBI                  5111 non-null int64
SB                   5111 non-null int64
CS                   5111 non-null object
BB                   5111 non-null int64
SO                   5111 non-null int64
BA                   5111 non-null object
OBP                  5111 non-null object
SLG                  5111 non-null object
OPS                  5111

In [709]:
df_2018 = df[df['Year'] == 2018]
df_2018_team = df_2018[['Player_Name', 'Team']]
df_2018_pos = df_2018[['Player_Name', 'Pos']]

In [710]:
df_2018_pos

,Player_Name,Pos
1,Scott Alexander,1
11,Rich Hill,1
19,Daniel Hudson,1
28,Kenley Jansen,1
34,Alex Wood,1
45,Clayton Kershaw,1
48,Kenta Maeda,1
51,Ross Stripling,1
53,Pat Venditte,/1
58,Erik Goeddel,1


In [711]:
df_2018_pos['Pos'] = df_2018_pos['Pos'].str.replace('*', '')
df_2018_pos['Pos'] = df_2018_pos['Pos'].str.replace('/', '')
df_2018_pos['Pos'] = df_2018_pos['Pos'].str[0]

/Users/brendenrossin/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.
/Users/brendenrossin/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
/Users/brendenrossin/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See 

In [712]:
df_2018_pos

,Player_Name,Pos
1,Scott Alexander,1
11,Rich Hill,1
19,Daniel Hudson,1
28,Kenley Jansen,1
34,Alex Wood,1
45,Clayton Kershaw,1
48,Kenta Maeda,1
51,Ross Stripling,1
53,Pat Venditte,1
58,Erik Goeddel,1


In [713]:
df_2018_team['Team'].unique()

array(['LAD', 'TOT', 'TBR', 'STL', 'SDP', 'SFG', 'COL', 'LAA', 'WSN',
       'PIT', 'TOR', 'DET', 'NYM', 'MIA', 'CHW', 'MIN', 'CIN', 'TEX',
       'KCR', 'BAL', 'BOS', 'NYY', 'HOU', 'SEA', 'CHC', 'MIL', 'PHI',
       'OAK', 'ATL', 'CLE', 'ARI'], dtype=object)

In [714]:
df = df[df['Year'] > 2010]
df = df[df['Year'] < 2018]

In [715]:
df.loc[:, 'BA'] = df.loc[:, 'Hits'] / df.loc[:, 'At_Bats']
df.loc[:, 'OBP'] = (df.loc[:, 'Hits'] + df.loc[:, 'BB'] + df.loc[:, 'HBP']) / \
    (df.loc[:, 'At_Bats'] + df.loc[:, 'BB'] + df.loc[:, 'HBP'] + df.loc[:, 'SF'])
df.loc[:, 'SLG'] = df.loc[:, 'TB'] / df.loc[:, 'At_Bats']
df.loc[:, 'OPS'] = df.loc[:, 'OBP'] + df.loc[:, 'SLG']

In [716]:
df['BA']
df['OBP']
df['SLG']
df['OPS']

df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 3520 entries, 0 to 2269
Data columns (total 33 columns):
Player_Name          3520 non-null object
Year                 3520 non-null int64
Age                  3520 non-null int64
Team                 3520 non-null object
League               3520 non-null object
Games_Played         3520 non-null int64
Plate_Appearances    3520 non-null int64
At_Bats              3520 non-null int64
Runs                 3520 non-null int64
Hits                 3520 non-null int64
2B                   3520 non-null int64
3B                   3520 non-null int64
HR                   3520 non-null int64
RBI                  3520 non-null int64
SB                   3520 non-null int64
CS                   3520 non-null object
BB                   3520 non-null int64
SO                   3520 non-null int64
BA                   2850 non-null float64
OBP                  2856 non-null float64
SLG                  2850 non-null float64
OPS                  2

In [717]:
def make_float(x):
    try:
        return float(x)
    except:
        return None

In [718]:
df['Salary'] = df['Salary'].str.replace(',', '')
df['Salary'] = df['Salary'].str.replace('$', '')
df['Salary'] = df['Salary'].apply(make_float)

In [719]:
df['Pos'] = df['Pos'].str.replace('*', '')
df['Pos'] = df['Pos'].str.replace('/', '')
df['Pos'] = df['Pos'].str[0]

In [720]:
df['All_Star'] = df['Awards'].str.replace('AS', '1')
df['All_Star'] = df['All_Star'].str[0]
df['All_Star'] = df['All_Star'].apply(make_float)

In [721]:
mean_salary = df['Salary'].mean()
df['Adj_Salary'] = df['Salary'] / mean_salary

In [722]:
df2 = df.groupby(["Player_Name"])[['Games_Played', 'Plate_Appearances', 'At_Bats', 'Runs', 'Hits', '2B', '3B',
                                   'HR', 'RBI', 'BA', 'OBP', 'SLG', 'OPS', 'SB', 'BB', 'SO', 'TB', 'HBP', 'SF',
                                   'dWAR','Salary','Adj_Salary']].mean()

In [723]:
df2['Age'] = df.groupby(['Player_Name'])['Age'].max()

In [724]:
df2['All_star_count'] = df.groupby(['Player_Name'])['All_Star'].sum()

In [725]:
df2['Total_games_played'] = df.groupby(['Player_Name'])['Games_Played'].sum()

In [726]:
df2

,Games_Played,Plate_Appearances,At_Bats,Runs,Hits,2B,3B,HR,RBI,BA,...,SO,TB,HBP,SF,dWAR,Salary,Adj_Salary,Age,All_star_count,Total_games_played
Player_Name,,,,,,,,,,,,,,,,,,,,,
A.J. Cole,7.000000,8.666667,7.666667,0.000000,0.666667,0.000000,0.000000,0.000000,0.000000,0.047619,...,3.000000,0.666667,0.000000,0.000000,0.000000,555300.0,0.081579,25,0.0,21
A.J. Ellis,78.571429,282.714286,239.428571,24.000000,56.285714,9.857143,0.428571,6.142857,28.142857,0.233300,...,50.857143,85.428571,3.714286,2.142857,0.571429,1250000.0,0.183638,36,0.0,550
A.J. Minter,16.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,NaN,...,0.000000,0.000000,0.000000,0.000000,0.000000,555000.0,0.081535,23,0.0,16
A.J. Pollock,87.333333,341.166667,310.666667,51.000000,89.000000,20.500000,4.000000,8.833333,33.166667,0.273747,...,51.166667,144.000000,2.000000,2.000000,0.966667,7750000.0,1.138556,29,1.0,524
AJ Reed,23.500000,73.500000,64.000000,5.500000,10.000000,1.500000,0.000000,1.500000,4.000000,0.081967,...,24.500000,16.000000,0.000000,0.500000,-0.150000,NaN,NaN,24,0.0,47
Aaron Altherr,51.250000,201.250000,178.000000,26.500000,43.250000,10.250000,2.250000,7.000000,27.250000,0.177338,...,54.000000,79.000000,4.500000,0.750000,-0.075000,561000.0,0.082417,26,0.0,205
Aaron Bummer,2.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,NaN,...,0.000000,0.000000,0.000000,0.000000,0.000000,549000.0,0.080654,23,0.0,2
Aaron Hicks,91.600000,330.000000,289.400000,38.600000,67.000000,12.200000,1.400000,8.600000,32.200000,0.229163,...,68.200000,107.800000,1.400000,2.600000,0.420000,2825000.0,0.415022,27,0.0,458
Aaron Judge,91.000000,386.500000,313.000000,69.000000,84.500000,13.000000,1.500000,28.000000,62.000000,0.231352,...,125.000000,184.500000,3.000000,2.500000,0.050000,622300.0,0.091422,25,1.0,182


In [727]:
df2.reset_index(inplace=True)

df2 = pd.merge(df2, df_2018_team)

df2.set_index('Player_Name')

,Games_Played,Plate_Appearances,At_Bats,Runs,Hits,2B,3B,HR,RBI,BA,...,TB,HBP,SF,dWAR,Salary,Adj_Salary,Age,All_star_count,Total_games_played,Team
Player_Name,,,,,,,,,,,,,,,,,,,,,
A.J. Cole,7.000000,8.666667,7.666667,0.000000,0.666667,0.000000,0.000000,0.000000,0.000000,0.047619,...,0.666667,0.000000,0.000000,0.000000,555300.0,0.081579,25,0.0,21,TOT
A.J. Ellis,78.571429,282.714286,239.428571,24.000000,56.285714,9.857143,0.428571,6.142857,28.142857,0.233300,...,85.428571,3.714286,2.142857,0.571429,1250000.0,0.183638,36,0.0,550,SDP
A.J. Minter,16.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,NaN,...,0.000000,0.000000,0.000000,0.000000,555000.0,0.081535,23,0.0,16,ATL
A.J. Pollock,87.333333,341.166667,310.666667,51.000000,89.000000,20.500000,4.000000,8.833333,33.166667,0.273747,...,144.000000,2.000000,2.000000,0.966667,7750000.0,1.138556,29,1.0,524,ARI
AJ Reed,23.500000,73.500000,64.000000,5.500000,10.000000,1.500000,0.000000,1.500000,4.000000,0.081967,...,16.000000,0.000000,0.500000,-0.150000,NaN,NaN,24,0.0,47,HOU
Aaron Altherr,51.250000,201.250000,178.000000,26.500000,43.250000,10.250000,2.250000,7.000000,27.250000,0.177338,...,79.000000,4.500000,0.750000,-0.075000,561000.0,0.082417,26,0.0,205,PHI
Aaron Bummer,2.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,NaN,...,0.000000,0.000000,0.000000,0.000000,549000.0,0.080654,23,0.0,2,CHW
Aaron Hicks,91.600000,330.000000,289.400000,38.600000,67.000000,12.200000,1.400000,8.600000,32.200000,0.229163,...,107.800000,1.400000,2.600000,0.420000,2825000.0,0.415022,27,0.0,458,NYY
Aaron Judge,91.000000,386.500000,313.000000,69.000000,84.500000,13.000000,1.500000,28.000000,62.000000,0.231352,...,184.500000,3.000000,2.500000,0.050000,622300.0,0.091422,25,1.0,182,NYY


In [728]:
df2.reset_index(inplace=True)

df2 = pd.merge(df2, df_2018_pos)

df2 = df2.set_index('Player_Name')

In [729]:
# DF3 AND DF4 ARE JUST USED TO CREATE POSITION COLUMN

In [730]:
df.Team.unique

<bound method Series.unique of 0       KCR
7       BOS
8       CLE
9       TOT
10      LAD
13      ARI
14      ARI
15      ARI
16      ARI
17      ARI
18      PIT
21      LAD
22      LAD
23      LAD
24      LAD
25      LAD
26      LAD
27      LAD
29      ATL
30      ATL
31      TOT
32      LAD
33      LAD
38      LAD
39      LAD
40      LAD
41      LAD
42      LAD
43      LAD
44      LAD
       ... 
2228    ATL
2229    ARI
2230    ARI
2234    BOS
2235    BOS
2236    BOS
2237    BOS
2238    PHI
2240    BOS
2241    BOS
2242    BOS
2245    KCR
2246    KCR
2247    KCR
2248    KCR
2249    KCR
2250    KCR
2251    SEA
2253    ARI
2254    ARI
2256    ARI
2259    ARI
2260    ARI
2262    ARI
2263    ARI
2264    ARI
2266    ARI
2267    BAL
2268    BAL
2269    ARI
Name: Team, Length: 3520, dtype: object>

In [731]:
df2.reset_index(inplace=True)

def remove_accents(word):
    return ''.join((c for c in unicodedata.normalize('NFD', word) if unicodedata.category(c) != 'Mn'))

df2['Player_Name'] = df2['Player_Name'].apply(remove_accents)

In [732]:
bautista = df2[df2['Player_Name'] == 'Jose Bautista']

bautista

,Player_Name,index,Games_Played,Plate_Appearances,At_Bats,Runs,Hits,2B,3B,HR,...,HBP,SF,dWAR,Salary,Adj_Salary,Age,All_star_count,Total_games_played,Team,Pos
458,Jose Bautista,456,134.285714,589.142857,486.142857,88.571429,123.428571,24.142857,0.857143,31.142857,...,5.428571,5.285714,-0.671429,500000.0,0.073455,36,5.0,940,TOT,9


In [733]:
markakis = df2[df2['Player_Name'] == 'Nick Markakis']

markakis

,Player_Name,index,Games_Played,Plate_Appearances,At_Bats,Runs,Hits,2B,3B,HR,...,HBP,SF,dWAR,Salary,Adj_Salary,Age,All_star_count,Total_games_played,Team,Pos
631,Nick Markakis,629,150.428571,662.428571,591.571429,73.857143,165.857143,32.142857,1.0,10.857143,...,4.571429,4.857143,-0.742857,2000000.0,0.293821,33,0.0,1053,ATL,9


In [734]:
plouffe = df2[df2['Player_Name'] == 'Trevor Plouffe']

plouffe

,Player_Name,index,Games_Played,Plate_Appearances,At_Bats,Runs,Hits,2B,3B,HR,...,HBP,SF,dWAR,Salary,Adj_Salary,Age,All_star_count,Total_games_played,Team,Pos
774,Trevor Plouffe,772,114.428571,454.0,411.428571,50.857143,100.142857,22.0,1.428571,14.714286,...,3.571429,3.142857,-0.128571,100000.0,0.014691,31,0.0,801,PHI,9


In [735]:
# fix errors


df2.reset_index(inplace=True)

## remove joey bats

bautista['Salary'] = np.nan

bautista['Salary']
print(bautista['Adj_Salary'])


df2[df2['Player_Name'] == 'Jose Bautista'] = bautista

df2[df2['Player_Name'] == 'Jose Bautista']

plouffe['Salary'] = np.nan

plouffe['Salary']



df2[df2['Player_Name'] == 'Trevor Plouffe'] = plouffe

## fix pablo sandoval

pablo = df2[df2['Player_Name'] == 'Pablo Sandoval']

pablo['Salary'] = 18600000

markakis['Salary'] = 11000000

df2[df2['Player_Name'] == 'Nick Markakis'] = markakis

pablo['Salary']

df2[df2['Player_Name'] == 'Pablo Sandoval'] = pablo

## fix ji-man choi

jiman = df2[df2['Player_Name'] == 'Ji-Man Choi']

jiman['Team'] = 'TBR'

jiman['Team']

df2[df2['Player_Name'] == 'Ji-Man Choi'] = jiman

/Users/brendenrossin/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


458    0.073455
Name: Adj_Salary, dtype: float64


/Users/brendenrossin/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/Users/brendenrossin/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:30: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/Users/brendenrossin/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:32: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.py

In [736]:
markakis

,Player_Name,index,Games_Played,Plate_Appearances,At_Bats,Runs,Hits,2B,3B,HR,...,HBP,SF,dWAR,Salary,Adj_Salary,Age,All_star_count,Total_games_played,Team,Pos
631,Nick Markakis,629,150.428571,662.428571,591.571429,73.857143,165.857143,32.142857,1.0,10.857143,...,4.571429,4.857143,-0.742857,11000000,0.293821,33,0.0,1053,ATL,9


In [737]:
df2['Adj_Salary'] = df2['Salary'] / mean_salary

In [738]:
df2[df2['Player_Name'] == 'Jose Bautista']

,level_0,Player_Name,index,Games_Played,Plate_Appearances,At_Bats,Runs,Hits,2B,3B,...,HBP,SF,dWAR,Salary,Adj_Salary,Age,All_star_count,Total_games_played,Team,Pos
458,NaN,Jose Bautista,456,134.285714,589.142857,486.142857,88.571429,123.428571,24.142857,0.857143,...,5.428571,5.285714,-0.671429,NaN,NaN,36,5.0,940,TOT,9


In [739]:
df2[df2['Player_Name'] == 'Pablo Sandoval']

,level_0,Player_Name,index,Games_Played,Plate_Appearances,At_Bats,Runs,Hits,2B,3B,...,HBP,SF,dWAR,Salary,Adj_Salary,Age,All_star_count,Total_games_played,Team,Pos
649,649.0,Pablo Sandoval,647,104.428571,417.285714,381.428571,43.428571,104.0,20.0,1.571429,...,2.714286,4.571429,3.172066e-17,18600000.0,2.732533,30,2.0,731,SFG,5


In [740]:
df2[df2['Player_Name'] == 'Ji-Man Choi']

,level_0,Player_Name,index,Games_Played,Plate_Appearances,At_Bats,Runs,Hits,2B,3B,...,HBP,SF,dWAR,Salary,Adj_Salary,Age,All_star_count,Total_games_played,Team,Pos
404,404.0,Ji-Man Choi,402,30.0,73.5,63.5,5.5,11.5,2.5,0.0,...,0.0,1.0,-0.15,850000.0,0.124874,26,0.0,60,TBR,D


In [741]:
df2.set_index('Player_Name', inplace=True)

In [742]:
## MAKE SURE THIS INDEX IS THE SAME WHEN I ADD MORE DATA



In [743]:
df2

,level_0,index,Games_Played,Plate_Appearances,At_Bats,Runs,Hits,2B,3B,HR,...,HBP,SF,dWAR,Salary,Adj_Salary,Age,All_star_count,Total_games_played,Team,Pos
Player_Name,,,,,,,,,,,,,,,,,,,,,
A.J. Cole,0.0,0,7.000000,8.666667,7.666667,0.000000,0.666667,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,555300.0,0.081579,25,0.0,21,TOT,1
A.J. Ellis,1.0,1,78.571429,282.714286,239.428571,24.000000,56.285714,9.857143,0.428571,6.142857,...,3.714286,2.142857,0.571429,1250000.0,0.183638,36,0.0,550,SDP,2
A.J. Minter,2.0,2,16.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,555000.0,0.081535,23,0.0,16,ATL,1
A.J. Pollock,3.0,3,87.333333,341.166667,310.666667,51.000000,89.000000,20.500000,4.000000,8.833333,...,2.000000,2.000000,0.966667,7750000.0,1.138556,29,1.0,524,ARI,8
AJ Reed,4.0,4,23.500000,73.500000,64.000000,5.500000,10.000000,1.500000,0.000000,1.500000,...,0.000000,0.500000,-0.150000,NaN,NaN,24,0.0,47,HOU,3
Aaron Altherr,5.0,5,51.250000,201.250000,178.000000,26.500000,43.250000,10.250000,2.250000,7.000000,...,4.500000,0.750000,-0.075000,561000.0,0.082417,26,0.0,205,PHI,9
Aaron Bummer,6.0,6,2.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,549000.0,0.080654,23,0.0,2,CHW,1
Aaron Hicks,7.0,7,91.600000,330.000000,289.400000,38.600000,67.000000,12.200000,1.400000,8.600000,...,1.400000,2.600000,0.420000,2825000.0,0.415022,27,0.0,458,NYY,8
Aaron Judge,8.0,8,91.000000,386.500000,313.000000,69.000000,84.500000,13.000000,1.500000,28.000000,...,3.000000,2.500000,0.050000,622300.0,0.091422,25,1.0,182,NYY,9


In [744]:
df2 = df2.drop(['index'],1)

In [745]:
df2 = df2.rename(index=str, columns={"level_0": "index"})

In [746]:
df2.info()

<class 'pandas.core.frame.DataFrame'>
Index: 847 entries, A.J. Cole to Alex Claudio
Data columns (total 28 columns):
index                 844 non-null float64
Games_Played          847 non-null float64
Plate_Appearances     847 non-null float64
At_Bats               847 non-null float64
Runs                  847 non-null float64
Hits                  847 non-null float64
2B                    847 non-null float64
3B                    847 non-null float64
HR                    847 non-null float64
RBI                   847 non-null float64
BA                    753 non-null float64
OBP                   756 non-null float64
SLG                   753 non-null float64
OPS                   753 non-null float64
SB                    847 non-null float64
BB                    847 non-null float64
SO                    847 non-null float64
TB                    847 non-null float64
HBP                   847 non-null float64
SF                    847 non-null float64
dWAR                  8

In [747]:
df2 = df2.dropna()

In [748]:
df2

,index,Games_Played,Plate_Appearances,At_Bats,Runs,Hits,2B,3B,HR,RBI,...,HBP,SF,dWAR,Salary,Adj_Salary,Age,All_star_count,Total_games_played,Team,Pos
Player_Name,,,,,,,,,,,,,,,,,,,,,
A.J. Cole,0.0,7.000000,8.666667,7.666667,0.000000,0.666667,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,555300.0,0.081579,25,0.0,21,TOT,1
A.J. Ellis,1.0,78.571429,282.714286,239.428571,24.000000,56.285714,9.857143,0.428571,6.142857,28.142857,...,3.714286,2.142857,0.571429,1250000.0,0.183638,36,0.0,550,SDP,2
A.J. Pollock,3.0,87.333333,341.166667,310.666667,51.000000,89.000000,20.500000,4.000000,8.833333,33.166667,...,2.000000,2.000000,0.966667,7750000.0,1.138556,29,1.0,524,ARI,8
Aaron Altherr,5.0,51.250000,201.250000,178.000000,26.500000,43.250000,10.250000,2.250000,7.000000,27.250000,...,4.500000,0.750000,-0.075000,561000.0,0.082417,26,0.0,205,PHI,9
Aaron Hicks,7.0,91.600000,330.000000,289.400000,38.600000,67.000000,12.200000,1.400000,8.600000,32.200000,...,1.400000,2.600000,0.420000,2825000.0,0.415022,27,0.0,458,NYY,8
Aaron Judge,8.0,91.000000,386.500000,313.000000,69.000000,84.500000,13.000000,1.500000,28.000000,62.000000,...,3.000000,2.500000,0.050000,622300.0,0.091422,25,1.0,182,NYY,9
Aaron Loup,9.0,3.200000,0.200000,0.200000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,1812500.0,0.266275,29,0.0,16,TOR,1
Aaron Nola,10.0,18.666667,38.333333,33.666667,1.333333,2.333333,0.333333,0.000000,0.000000,0.666667,...,0.333333,0.000000,0.000000,573000.0,0.084180,24,0.0,56,PHI,1
Abraham Almonte,12.0,60.400000,189.800000,172.400000,23.000000,42.400000,10.800000,2.000000,2.800000,16.800000,...,0.600000,1.800000,0.060000,825000.0,0.121201,28,0.0,302,KCR,8


In [749]:
with open('all_top40.pkl', 'wb') as picklefile:
    pickle.dump(df2, picklefile)

In [624]:
df2.info()

<class 'pandas.core.frame.DataFrame'>
Index: 598 entries, A.J. Cole to Zack Wheeler
Data columns (total 28 columns):
index                 598 non-null float64
Games_Played          598 non-null float64
Plate_Appearances     598 non-null float64
At_Bats               598 non-null float64
Runs                  598 non-null float64
Hits                  598 non-null float64
2B                    598 non-null float64
3B                    598 non-null float64
HR                    598 non-null float64
RBI                   598 non-null float64
BA                    598 non-null float64
OBP                   598 non-null float64
SLG                   598 non-null float64
OPS                   598 non-null float64
SB                    598 non-null float64
BB                    598 non-null float64
SO                    598 non-null float64
TB                    598 non-null float64
HBP                   598 non-null float64
SF                    598 non-null float64
dWAR                  5

In [625]:
df2[df2['Salary'] == 11000000]

,index,Games_Played,Plate_Appearances,At_Bats,Runs,Hits,2B,3B,HR,RBI,...,HBP,SF,dWAR,Salary,Adj_Salary,Age,All_star_count,Total_games_played,Team,Pos
Player_Name,,,,,,,,,,,,,,,,,,,,,
Andrelton Simmons,48.0,130.166667,521.500000,480.666667,53.666667,127.000000,22.666667,3.000000,8.166667,46.833333,...,2.500000,3.833333,3.683333,11000000.0,1.616014,27,0.0,781,LAA,6
Denard Span,240.0,118.714286,523.285714,476.428571,65.428571,133.714286,26.714286,5.428571,6.142857,37.000000,...,1.714286,1.428571,0.042857,11000000.0,1.616014,33,0.0,831,TOT,7
Ian Kinsler,338.0,150.714286,680.142857,613.857143,101.714286,165.428571,33.714286,4.142857,20.285714,74.428571,...,7.714286,3.857143,1.771429,11000000.0,1.616014,35,2.0,1055,LAA,4
Jay Bruce,385.0,151.285714,627.714286,562.571429,80.142857,138.428571,31.000000,3.000000,29.857143,94.000000,...,2.857143,4.857143,-0.471429,11000000.0,1.616014,30,3.0,1059,NYM,9
Kendrys Morales,488.0,141.666667,574.166667,522.833333,61.000000,138.333333,28.333333,0.500000,22.166667,79.833333,...,5.333333,3.333333,-1.166667,11000000.0,1.616014,34,0.0,850,TOR,D


In [496]:
df2.corr()

,Games_Played,Plate_Appearances,At_Bats,Runs,Hits,2B,3B,HR,RBI,BA,...,Intercept,Catcher,First_Base,Second_Base,Third_Base,Shortstop,Left_Field,Center_Field,Right_Field,DH
Games_Played,1.000000,0.966533,0.966348,0.945866,0.957274,0.945161,0.708242,0.836525,0.926197,0.517860,...,NaN,0.033058,0.234522,0.204347,0.219303,0.199611,0.173640,0.167466,0.207094,0.154461
Plate_Appearances,0.966533,1.000000,0.999083,0.984366,0.992998,0.979235,0.732559,0.873336,0.962062,0.536980,...,NaN,0.039565,0.239948,0.206361,0.222717,0.205743,0.180635,0.148050,0.222092,0.163528
At_Bats,0.966348,0.999083,1.000000,0.980465,0.994153,0.978834,0.738166,0.866356,0.958739,0.538878,...,NaN,0.038719,0.232820,0.211417,0.223020,0.214470,0.179980,0.151116,0.218189,0.162757
Runs,0.945866,0.984366,0.980465,1.000000,0.981882,0.970256,0.746599,0.894875,0.960580,0.525103,...,NaN,-0.012753,0.238421,0.203416,0.211435,0.177093,0.198599,0.168625,0.236248,0.158508
Hits,0.957274,0.992998,0.994153,0.981882,1.000000,0.983038,0.740811,0.857323,0.957042,0.542689,...,NaN,0.024180,0.236765,0.215267,0.220856,0.209541,0.174927,0.144498,0.213609,0.158000
2B,0.945161,0.979235,0.978834,0.970256,0.983038,1.000000,0.703093,0.861580,0.955683,0.531699,...,NaN,0.022192,0.253554,0.203354,0.250410,0.181869,0.175190,0.122665,0.213085,0.151671
3B,0.708242,0.732559,0.738166,0.746599,0.740811,0.703093,1.000000,0.516880,0.623654,0.426400,...,NaN,-0.101655,0.047925,0.234823,0.113824,0.255365,0.208407,0.274852,0.218497,-0.008680
HR,0.836525,0.873336,0.866356,0.894875,0.857323,0.861580,0.516880,1.000000,0.955563,0.444655,...,NaN,0.013014,0.307795,0.064360,0.216673,0.075227,0.186221,0.073317,0.208501,0.267745
RBI,0.926197,0.962062,0.958739,0.960580,0.957042,0.955683,0.623654,0.955563,1.000000,0.508099,...,NaN,0.038273,0.292241,0.130604,0.238231,0.141510,0.177594,0.085793,0.217021,0.221615
BA,0.517860,0.536980,0.538878,0.525103,0.542689,0.531699,0.426400,0.444655,0.508099,1.000000,...,NaN,0.098170,0.146685,0.158999,0.160190,0.139225,0.140732,0.112834,0.146197,0.097171


In [497]:
df2

,Games_Played,Plate_Appearances,At_Bats,Runs,Hits,2B,3B,HR,RBI,BA,...,Intercept,Catcher,First_Base,Second_Base,Third_Base,Shortstop,Left_Field,Center_Field,Right_Field,DH
Player_Name,,,,,,,,,,,,,,,,,,,,,
A.J. Cole,7.000000,8.666667,7.666667,0.000000,0.666667,0.000000,0.000000,0.000000,0.000000,0.047619,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
A.J. Ellis,78.571429,282.714286,239.428571,24.000000,56.285714,9.857143,0.428571,6.142857,28.142857,0.233300,...,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
A.J. Pollock,87.333333,341.166667,310.666667,51.000000,89.000000,20.500000,4.000000,8.833333,33.166667,0.273747,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
Aaron Altherr,51.250000,201.250000,178.000000,26.500000,43.250000,10.250000,2.250000,7.000000,27.250000,0.177338,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
Aaron Hicks,91.600000,330.000000,289.400000,38.600000,67.000000,12.200000,1.400000,8.600000,32.200000,0.229163,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
Aaron Judge,91.000000,386.500000,313.000000,69.000000,84.500000,13.000000,1.500000,28.000000,62.000000,0.231352,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
Aaron Loup,3.200000,0.200000,0.200000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Aaron Nola,18.666667,38.333333,33.666667,1.333333,2.333333,0.333333,0.000000,0.000000,0.666667,0.067874,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Abraham Almonte,60.400000,189.800000,172.400000,23.000000,42.400000,10.800000,2.000000,2.800000,16.800000,0.248115,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0


In [498]:
X=patsy.dmatrix('Pos',data=df2,return_type='dataframe')
X.head()

,Intercept,Pos[T.2],Pos[T.3],Pos[T.4],Pos[T.5],Pos[T.6],Pos[T.7],Pos[T.8],Pos[T.9],Pos[T.D]
Player_Name,,,,,,,,,,
A.J. Cole,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
A.J. Ellis,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
A.J. Pollock,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
Aaron Altherr,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
Aaron Hicks,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0


In [499]:
df2=df2.join(X)

ValueError: columns overlap but no suffix specified: Index(['Intercept'], dtype='object')

In [ ]:
df2['Catcher'] = df2['Pos[T.2]']
df2['First_Base'] = df2['Pos[T.3]']
df2['Second_Base'] = df2['Pos[T.4]']
df2['Third_Base'] = df2['Pos[T.5]']
df2['Shortstop'] = df2['Pos[T.6]']
df2['Left_Field'] = df2['Pos[T.7]']
df2['Center_Field'] = df2['Pos[T.8]']
df2['Right_Field'] = df2['Pos[T.9]']
df2['DH'] = df2['Pos[T.D]']

df2 = df2.drop(['Pos[T.2]','Pos[T.3]','Pos[T.4]','Pos[T.5]','Pos[T.6]','Pos[T.7]','Pos[T.8]','Pos[T.9]','Pos[T.D]'],1)

In [ ]:
df2

In [ ]:
df2.info()

In [ ]:
batters_only = df2[df2['Pos'] != '1']

In [ ]:
batters_only = batters_only.rename(index=str, columns={"2B": "Doubles", "3B": "Triples"})

In [ ]:
batters_only.info()

In [ ]:
batters_only.corr()

In [ ]:
sns.stripplot(x="Pos", y="Adj_Salary", data=batters_only)

In [ ]:
salary_df = batters_only.sort_values(by='Adj_Salary', ascending=False)
salary_df[:10]

In [ ]:
batters_only[batters_only['index'] == 169]['Adj_Salary']

In [ ]:
batters_only[batters_only['index'] == 411]['Adj_Salary']

In [ ]:
batters_only_df = batters_only[['Runs', 'Hits', 'Doubles', 'Triples', 'HR', 'RBI', 'BA', 'OBP', 'SLG',
                                'OPS', 'SB', 'TB', 'SF', 'Adj_Salary', 'Age', 'Total_games_played',
                                'dWAR','All_star_count','Catcher','First_Base','Second_Base','Third_Base',
                                'Shortstop','Left_Field','Center_Field','Right_Field','DH','Team']]

In [ ]:
#sns.pairplot(batters_only_df, size=1.2, aspect=1.5)

In [ ]:
batters_only_df['log_salary'] = np.log(batters_only_df.Adj_Salary)

batters_only_df.log_salary.hist()

In [ ]:
batters_only_df.corr()

In [ ]:
batters_only_df['Team'].unique()

In [ ]:
batters_only_df.corr()

In [ ]:
plt.scatter(batters_only_df['BA'], batters_only_df['Adj_Salary'])

In [ ]:
## Remove players below Mendoza line

batters_only_df = batters_only_df[batters_only_df['BA'] > 0.2]

plt.scatter(batters_only_df['BA'], batters_only_df['Adj_Salary'])

In [ ]:
y, X = patsy.dmatrices('Adj_Salary ~ Runs + Hits + Doubles + HR + RBI + BA + OBP + SLG + OPS + SB + TB + SF + Age + Total_games_played + All_star_count + dWAR',
                       data=batters_only_df, return_type="dataframe")

model = sm.OLS(y, X)
fit = model.fit()
fit.summary()

In [ ]:
diagnostic_plots.diagnostic_plots(batters_only_df.drop(
    'Adj_Salary', axis=1), batters_only_df['Adj_Salary'], fit)

In [ ]:
y, X = patsy.dmatrices('log_salary ~ Runs + Hits + Doubles + HR + RBI + BA + OBP + SLG + OPS + SB + TB + SF + Age + Total_games_played + All_star_count + dWAR',
                       data=batters_only_df, return_type="dataframe")

model2 = sm.OLS(y, X)
fit2 = model2.fit()
fit2.summary()

In [ ]:
diagnostic_plots.diagnostic_plots(batters_only_df.drop(
    'log_salary', axis=1), batters_only_df['log_salary'], fit2)

In [ ]:
y, X = patsy.dmatrices('log_salary ~ Runs + Hits + Doubles + RBI + OBP + SLG + OPS + SB + TB + SF + Age + Total_games_played + All_star_count + dWAR',
                       data=batters_only_df, return_type="dataframe")

model3 = sm.OLS(y, X)

fit3 = model3.fit()

fit3.summary()

In [ ]:
diagnostic_plots.diagnostic_plots(batters_only_df.drop(
    'log_salary', axis=1), batters_only_df['log_salary'], fit3)

In [ ]:
y, X = patsy.dmatrices('log_salary ~ Runs + Doubles + RBI + OBP + SLG + OPS + SB + TB + SF + Age + Total_games_played + All_star_count + dWAR',
                       data=batters_only_df, return_type="dataframe")

model4 = sm.OLS(y, X)

fit4 = model4.fit()

fit4.summary()

In [ ]:
diagnostic_plots.diagnostic_plots(batters_only_df.drop(
    'log_salary', axis=1), batters_only_df['log_salary'], fit4)

In [ ]:
y, X = patsy.dmatrices('log_salary ~ Hits + HR + RBI + BA + OPS + SB + TB + Age + dWAR + Total_games_played',
                       data=batters_only_df, return_type="dataframe")

model5 = sm.OLS(y, X)

fit5 = model5.fit()

fit5.summary()

In [ ]:
diagnostic_plots.diagnostic_plots(batters_only_df.drop(
    'log_salary', axis=1), batters_only_df['log_salary'], fit5)

In [ ]:
fit5.resid.plot(style='o', figsize=(12, 8))

In [ ]:
y, X = patsy.dmatrices('log_salary ~ Runs + Hits + Doubles + HR + RBI + BA + OBP + SLG + OPS + SB + TB + SF + Age + Total_games_played + All_star_count + dWAR + Catcher + First_Base + Second_Base + Third_Base + Shortstop + Left_Field + Center_Field + Right_Field + DH',
                       data=batters_only_df, return_type="dataframe")

model6 = sm.OLS(y, X)

fit6 = model6.fit()

fit6.summary()

In [ ]:
diagnostic_plots.diagnostic_plots(batters_only_df.drop(
    'log_salary', axis=1), batters_only_df['log_salary'], fit6)

In [ ]:
batters_only_df['rootASC'] = batters_only_df['All_star_count']**0.5

In [ ]:
y, X = patsy.dmatrices('log_salary ~ Doubles + RBI + BA + SB + TB + Total_games_played + rootASC + dWAR + Catcher + First_Base + Second_Base + Third_Base + Shortstop + Left_Field + Center_Field + Right_Field',
                       data=batters_only_df, return_type="dataframe")

model7 = sm.OLS(y, X)

fit7 = model7.fit()

fit7.summary()

In [ ]:
diagnostic_plots.diagnostic_plots(batters_only_df.drop(
    'log_salary', axis=1), batters_only_df['log_salary'], fit7)

In [ ]:

#sns.pairplot(new_df, size=1.2, aspect=1.5)

In [ ]:
X2=patsy.dmatrix('Team',data=batters_only_df,return_type='dataframe')
X2.head()

In [ ]:
batters_only_df=batters_only_df.join(X2)

In [ ]:
batters_only_df

In [ ]:
batters_only_df['LAA'] = batters_only_df['Team[T.LAA]']
batters_only_df['LAD'] = batters_only_df['Team[T.LAD]']
batters_only_df['PIT'] = batters_only_df['Team[T.PIT]']
batters_only_df['SDP'] = batters_only_df['Team[T.SDP]']
batters_only_df['STL'] = batters_only_df['Team[T.STL]']
batters_only_df['TBR'] = batters_only_df['Team[T.TBR]']
batters_only_df['TOR'] = batters_only_df['Team[T.TOR]']
batters_only_df['WSN'] = batters_only_df['Team[T.WSN]']
batters_only_df['SFG'] = batters_only_df['Team[T.SFG]']
batters_only_df['DET'] = batters_only_df['Team[T.DET]']
batters_only_df['NYM'] = batters_only_df['Team[T.NYM]']
batters_only_df['MIA'] = batters_only_df['Team[T.MIA]']


batters_only_df = batters_only_df.drop(['Team[T.LAA]','Team[T.LAD]','Team[T.PIT]',
                                        'Team[T.SDP]','Team[T.STL]','Team[T.TBR]',
                                        'Team[T.TOR]','Team[T.WSN]','Team[T.SFG]',
                                        'Team[T.DET]','Team[T.NYM]','Team[T.MIA]'],1)

In [ ]:
## change column Team[] names to team name

batters_only_df['ATL'] = batters_only_df['Team[T.ATL]']
batters_only_df['BAL'] = batters_only_df['Team[T.BAL]']
batters_only_df['BOS'] = batters_only_df['Team[T.BOS]']
batters_only_df['CHC'] = batters_only_df['Team[T.CHC]']
batters_only_df['CHW'] = batters_only_df['Team[T.CHW]']
batters_only_df['CIN'] = batters_only_df['Team[T.CIN]']
batters_only_df['CLE'] = batters_only_df['Team[T.CLE]']
batters_only_df['COL'] = batters_only_df['Team[T.COL]']
batters_only_df['HOU'] = batters_only_df['Team[T.HOU]']
batters_only_df['KCR'] = batters_only_df['Team[T.KCR]']
batters_only_df['MIL'] = batters_only_df['Team[T.MIL]']
batters_only_df['MIN'] = batters_only_df['Team[T.MIN]']
batters_only_df['NYY'] = batters_only_df['Team[T.NYY]']
batters_only_df['OAK'] = batters_only_df['Team[T.OAK]']
batters_only_df['PHI'] = batters_only_df['Team[T.PHI]']
batters_only_df['SEA'] = batters_only_df['Team[T.SEA]']
batters_only_df['TEX'] = batters_only_df['Team[T.TEX]']
batters_only_df['TOT'] = batters_only_df['Team[T.TOT]']


batters_only_df = batters_only_df.drop(['Team[T.ATL]','Team[T.BAL]','Team[T.BOS]',
                                        'Team[T.CHC]','Team[T.CHW]','Team[T.CIN]',
                                        'Team[T.CLE]','Team[T.COL]','Team[T.HOU]',
                                        'Team[T.KCR]','Team[T.MIL]','Team[T.MIN]',
                                        'Team[T.NYY]','Team[T.OAK]','Team[T.PHI]',
                                        'Team[T.SEA]','Team[T.TEX]','Team[T.TOT]',
                                        'Team'],1)

In [ ]:
batters_only_df.corr()

In [ ]:
y, X = patsy.dmatrices('log_salary ~ Hits + RBI + OPS + SB + Age + Total_games_played +'
                       'First_Base + Second_Base + Shortstop + Left_Field + Right_Field +'
                       'LAA + LAD + PIT + SDP + STL + TBR + TOR + WSN + SFG + DET + NYM + MIA',
                       data=batters_only_df, return_type="dataframe")

model12 = sm.OLS(y, X)

fit12 = model12.fit()

fit12.summary()

In [ ]:
diagnostic_plots.diagnostic_plots(batters_only_df.drop(
    'log_salary', axis=1), batters_only_df['log_salary'], fit12)

In [ ]:
y, X = patsy.dmatrices('log_salary ~ Hits + RBI + OPS + SB + Age + Total_games_played +'
                       'First_Base + Second_Base + Shortstop + Left_Field + Right_Field +'
                       'LAA + PIT + WSN',
                       data=batters_only_df, return_type="dataframe")

model13 = sm.OLS(y, X)

fit13 = model13.fit()

fit13.summary()

In [ ]:
## NORMALIZE NON-CATEGORICAL VALUES

In [ ]:
no_categorical = batters_only_df[['Runs', 'Hits', 'Doubles', 'Triples', 'HR', 'RBI',
                                  'BA', 'OBP', 'SLG', 'OPS', 'SB', 'TB', 'SF', 'Adj_Salary',
                                  'Age', 'Total_games_played', 'dWAR', 'All_star_count', 'log_salary']]

In [ ]:
no_categorical

In [ ]:
from sklearn import preprocessing
normalized_categorical = preprocessing.normalize(no_categorical, axis=0)

In [ ]:
normalized_categorical=pd.DataFrame(normalized_categorical,columns=no_categorical.columns)

In [ ]:
normalized_categorical

In [ ]:
no_categorical.reset_index(inplace=True)

In [ ]:
no_categorical

In [ ]:
normalized_categorical['Player_Name'] = no_categorical['Player_Name']

In [ ]:
normalized_categorical

In [ ]:
batters_only_df.columns

In [ ]:
categorical_data = batters_only_df[['Catcher','First_Base','Second_Base','Third_Base','Shortstop',
                                    'Left_Field', 'Right_Field','Center_Field','DH','LAA', 'LAD',
                                    'PIT', 'SDP', 'STL', 'TBR','TOR','WSN','SFG','DET','NYM','MIA',
                                    'ATL', 'BAL', 'BOS', 'CHC', 'CHW', 'CIN', 'CLE', 'COL','HOU',
                                    'KCR', 'MIL', 'MIN', 'NYY', 'OAK','PHI','SEA','TEX','TOT']]

In [ ]:
categorical_data

In [ ]:
categorical_data.reset_index(inplace=True)

In [ ]:
norm_batters = pd.merge(normalized_categorical, categorical_data)

In [ ]:
norm_batters = norm_batters.set_index('Player_Name')

In [ ]:
norm_batters.describe()

In [ ]:
salary_df = norm_batters.sort_values(by='Adj_Salary', ascending=False)
salary_df[:10]

In [ ]:
y = norm_batters.log_salary
X = norm_batters.drop(['log_salary','Adj_Salary'],1)

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y)

model.fit(X_train,y_train)
model.score(X_test,y_test)